In [1]:
datasets=["pbmc_cite",
        "openproblems_2022_multi_atac2gex_subset",
        "openproblems_2022_cite_gex2adt_subset",
        "5k_pbmc_subset",
        "10k_pbmc",
        "GSE117089_mouse_gex2atac",
        "GSE117089_sciCAR_gex2atac",
        "GSE127064_AdBrain_gex2atac",
        "GSE127064_p0Brain_gex2atac",
        "GSE140203_3T3_HG19_atac2gex",
        "GSE140203_3T3_MM10_atac2gex",
        "GSE140203_12878.rep2_atac2gex",
        "GSE140203_12878.rep3_atac2gex",
        "GSE140203_K562_HG19_atac2gex",
        "GSE140203_K562_MM10_atac2gex",
        "GSE140203_LUNG_atac2gex",]

In [2]:
subSet=-1

In [3]:
CMAE_scores=[]

"""Main functionality for starting training.

This code is based on https://github.com/NVlabs/MUNIT.

"""
import argparse
import os
import random

import torch
from sklearn import preprocessing

from dance.datasets.multimodality import ModalityMatchingDataset
from dance.modules.multi_modality.match_modality.cmae import CMAE
from dance.utils import set_seed



rndseed = random.randint(0, 2147483647)
parser = argparse.ArgumentParser()
parser.add_argument("--output_path", type=str, default="./match_modality/output", help="outputs path")
parser.add_argument("-d", "--data_folder", default="../../../../data/modality_matching")
parser.add_argument("--resume", action="store_true")
parser.add_argument("-t", "--subtask", default="openproblems_bmmc_cite_phase2_rna")
parser.add_argument("-device", "--device", default="cpu")
parser.add_argument("-cpu", "--cpus", default=1, type=int)
parser.add_argument("-seed", "--rnd_seed", default=rndseed, type=int)
parser.add_argument("-pk", "--pickle_suffix", default="_lsi_input_pca_count.pkl")
parser.add_argument("--span", default=0.3, type=float)
parser.add_argument("--max_epochs", default=20, type=int, help="maximum number of training epochs")
parser.add_argument("--batch_size", default=30, type=int, help="batch size")
parser.add_argument("--log_data", default=True, type=bool, help="take a log1p of the data as input")
parser.add_argument("--normalize_data", default=True, type=bool,
                    help="normalize the data (after the log, if applicable)")
parser.add_argument("--weight_decay", default=1e-4, type=float, help="weight decay")
parser.add_argument("--beta1", default=0.5, type=float, help="Adam parameter")
parser.add_argument("--beta2", default=0.999, type=float, help="Adam parameter")
parser.add_argument("--init", default="kaiming", type=str,
                    help="initialization [gaussian/kaiming/xavier/orthogonal]")
parser.add_argument("--lr", default=1e-4, type=float, help="initial learning rate")
parser.add_argument("--lr_policy", default="step", type=str, help="learning rate scheduler")
parser.add_argument("--step_size", default=100000, type=int, help="how often to decay learning rate")
parser.add_argument("--gamma", default=0.5, type=float, help="how much to decay learning rate")
parser.add_argument("--gan_w", default=10, type=int, help="weight of adversarial loss")
parser.add_argument("--recon_x_w", default=10, type=int, help="weight of image reconstruction loss")
parser.add_argument("--recon_h_w", default=0, type=int, help="weight of hidden reconstruction loss")
parser.add_argument("--recon_kl_w", default=0, type=int, help="weight of KL loss for reconstruction")
parser.add_argument("--supervise", default=1, type=float, help="fraction to supervise")
parser.add_argument("--super_w", default=0.1, type=float, help="weight of supervision loss")
for dataset in datasets:
    opts = parser.parse_args(['--subtask',dataset,'--device','cuda:6','--span','1.0'])

    torch.set_num_threads(opts.cpus)
    rndseed = opts.rnd_seed
    device = opts.device
    set_seed(rndseed)

    # Setup logger and output folders
    output_directory = os.path.join(opts.output_path, "outputs")
    checkpoint_directory = os.path.join(output_directory, "checkpoints")
    os.makedirs(checkpoint_directory, exist_ok=True)

    # Preprocess and load data
    dataset = ModalityMatchingDataset(opts.subtask, root=opts.data_folder, preprocess="feature_selection",span=opts.span)
    data = dataset.load_data()

    # Prepare extra batch features and set data configs
    le = preprocessing.LabelEncoder()
    batch = le.fit_transform(data.mod["mod1"].obs["batch"])
    data.mod["mod1"].obsm["batch"] = batch
    data.set_config(feature_mod=["mod1", "mod2", "mod1"], label_mod="mod1", feature_channel=[None, None, "batch"],
                    label_channel="labels")

    # Obtain training and testing data
    (x_train, y_train, batch), _ = data.get_train_data(return_type="torch")
    (x_test, y_test, _), labels = data.get_test_data(return_type="torch")
    if subSet>0:
        batch=batch[:subSet]
        x_train=x_train[:subSet]
        y_train=y_train[:subSet]
        x_test=x_test[:subSet]
        y_test=y_test[:subSet]
        labels=labels[:subSet]
    batch = batch.long().to(device)
    x_train = x_train.float().to(device)
    y_train = y_train.float().to(device)
    x_test = x_test.float().to(device)
    y_test = y_test.float().to(device)
    labels = labels.long().to(device)
    

   

    config = vars(opts)
    # Some Fixed Settings
    config["input_dim_a"] = data.mod["mod1"].shape[1]
    config["input_dim_b"] = data.mod["mod2"].shape[1]
    config["resume"] = opts.resume
    config["num_of_classes"] = max(batch) + 1
    config["shared_layer"] = True
    config["gen"] = {
        "dim": 100,  # hidden layer
        "latent": 50,  # latent layer size
        "activ": "relu",
    }  # activation function [relu/lrelu/prelu/selu/tanh]
    config["dis"] = {
        "dim": 100,
        "norm": None,  # normalization layer [none/bn/in/ln]
        "activ": "lrelu",  # activation function [relu/lrelu/prelu/selu/tanh]
        "gan_type": "lsgan",
    }  # GAN loss [lsgan/nsgan]

    model = CMAE(config)
    model.to(device)

    model.fit(x_train, y_train, checkpoint_directory=checkpoint_directory)
    print(model.predict(x_test, y_test))
    score=model.score(x_test, y_test, labels)
    print(score)
    CMAE_scores.append(score)
    torch.cuda.empty_cache()

"""To reproduce CMAE on other samples, please refer to command lines belows:

GEX-ADT (subset):
python cmae.py --subtask openproblems_bmmc_cite_phase2_rna_subset --device cuda

GEX-ADT:
python cmae.py --subtask openproblems_bmmc_cite_phase2_rna --device cuda

GEX-ATAC:
python cmae.py --subtask openproblems_bmmc_multiome_phase2_rna --device cuda

"""


[INFO][2023-10-07 21:16:10,837][dance][set_seed] Setting global random seed to 1961419917
[INFO][2023-10-07 21:16:10,840][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/pbmc_cite/pbmc_cite.citeanti_dataset.output_train_mod1.h5ad
[INFO][2023-10-07 21:16:11,283][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/pbmc_cite/pbmc_cite.citeanti_dataset.output_train_mod2.h5ad
[INFO][2023-10-07 21:16:11,326][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/pbmc_cite/pbmc_cite.citeanti_dataset.output_train_sol.h5ad
[INFO][2023-10-07 21:16:11,372][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/pbmc_cite/pbmc_cite.citeanti_dataset.output_test_mod1.h5ad
[INFO][2023-10-07 21:16:11,588][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/pbmc_cite/pbmc_cite.citeanti_dataset.output_test_mod2.h5ad
[INFO][2023-10-07 21:16:11,614][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/pbmc_cite/pbmc_cite

Iteration:  0
Matching score: 0.0005693139974027872
Iteration:  1
Matching score: 0.0008539709961041808
Iteration:  2
Matching score: 0.0008539709961041808
Iteration:  3
Matching score: 0.0005693139974027872
Iteration:  4
Matching score: 0.002277255989611149
Iteration:  5
Matching score: 0.0054084830917418
Iteration:  6
Matching score: 0.005123825743794441
Iteration:  7
Matching score: 0.006547111086547375
Iteration:  8
Matching score: 0.00740108173340559
Iteration:  9
Matching score: 0.005977796856313944
Iteration:  10
Matching score: 0.009678337723016739
Iteration:  11
Matching score: 0.00996299460530281
Iteration:  12
Matching score: 0.012809565290808678
Iteration:  13
Matching score: 0.011101623065769672
Iteration:  14
Matching score: 0.013663535937666893
Iteration:  15
Matching score: 0.012809565290808678
Iteration:  16
Matching score: 0.011386279948055744
Iteration:  17
Matching score: 0.011670936830341816
Iteration:  18
Matching score: 0.012809565290808678
Iteration:  19
Matchin

[INFO][2023-10-07 21:23:05,999][dance][set_seed] Setting global random seed to 1961419917
[INFO][2023-10-07 21:23:06,001][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/openproblems_2022_multi_atac2gex_subset/openproblems_2022_multi_atac2gex_subset.open_dataset.output_train_mod1.h5ad


0.0062767756171524525


[INFO][2023-10-07 21:23:06,390][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/openproblems_2022_multi_atac2gex_subset/openproblems_2022_multi_atac2gex_subset.open_dataset.output_train_mod2.h5ad
[INFO][2023-10-07 21:23:06,621][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/openproblems_2022_multi_atac2gex_subset/openproblems_2022_multi_atac2gex_subset.open_dataset.output_train_sol.h5ad
[INFO][2023-10-07 21:23:06,665][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/openproblems_2022_multi_atac2gex_subset/openproblems_2022_multi_atac2gex_subset.open_dataset.output_test_mod1.h5ad
[INFO][2023-10-07 21:23:07,146][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/openproblems_2022_multi_atac2gex_subset/openproblems_2022_multi_atac2gex_subset.open_dataset.output_test_mod2.h5ad
[INFO][2023-10-07 21:23:07,366][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/openproblems_2022_multi_atac2gex_subset/open

...

[INFO][2023-10-07 21:47:40,644][dance][set_seed] Setting global random seed to 1961419917
[INFO][2023-10-07 21:47:40,646][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_train_mod1.h5ad


0.0021528524812310934


[INFO][2023-10-07 21:47:40,948][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_train_mod2.h5ad
[INFO][2023-10-07 21:47:40,981][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_train_sol.h5ad
[INFO][2023-10-07 21:47:40,994][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_test_mod1.h5ad
[INFO][2023-10-07 21:47:41,247][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_test_mod2.h5ad
[INFO][2023-10-07 21:47:41,278][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_test_sol.h5ad
[INFO][2023-10-07 21:47:46,902][dance][_maybe_preprocess] Preproc

Iteration:  0
Matching score: 0.0069444444961845875
Iteration:  1
Matching score: 0.02083333395421505
Iteration:  2
Matching score: 0.0069444444961845875
Iteration:  3
Matching score: 0.0069444444961845875
Iteration:  4
Matching score: 0.0069444444961845875
Iteration:  5
Matching score: 0.0069444444961845875
Iteration:  6
Matching score: 0.0069444444961845875
Iteration:  7
Matching score: 0.0069444444961845875
Iteration:  8
Matching score: 0.0069444444961845875
Iteration:  9
Matching score: 0.0069444444961845875
Iteration:  10
Matching score: 0.0069444444961845875
Iteration:  11
Matching score: 0.0069444444961845875
Iteration:  12
Matching score: 0.0069444444961845875
Iteration:  13
Matching score: 0.0069444444961845875
Iteration:  14
Matching score: 0.0069444444961845875
Iteration:  15
Matching score: 0.0069444444961845875
Iteration:  16
Matching score: 0.0069444444961845875
Iteration:  17
Matching score: 0.0069444444961845875
Iteration:  18
Matching score: 0.0069444444961845875
Itera

'To reproduce CMAE on other samples, please refer to command lines belows:\n\nGEX-ADT (subset):\npython cmae.py --subtask openproblems_bmmc_cite_phase2_rna_subset --device cuda\n\nGEX-ADT:\npython cmae.py --subtask openproblems_bmmc_cite_phase2_rna --device cuda\n\nGEX-ATAC:\npython cmae.py --subtask openproblems_bmmc_multiome_phase2_rna --device cuda\n\n'

In [4]:
CMAE_scores

[0.0062767756171524525,
 0.0005999999702908099,
 0.0005999999702908099,
 0.0009999999310821295,
 0.00033333332976326346,
 0.0,
 0.0,
 0.0003233106981497258,
 0.0013114754110574722,
 0.00159744406118989,
 0.0025125627871602774,
 0.001183431944809854,
 0.00013274923549033701,
 0.0,
 0.0021528524812310934,
 0.004878048785030842]

In [5]:
scMM_scores=[]
import argparse
import random
subSet=5000
import torch
import scanpy as sc
from dance.datasets.multimodality import ModalityMatchingDataset
from dance.modules.multi_modality.match_modality.scmm import MMVAE
from dance.utils import set_seed

rndseed = random.randint(0, 2147483647)
parser = argparse.ArgumentParser()
parser.add_argument("--output_path", type=str, default="./modality_matching/output", help="outputs path")
parser.add_argument("-d", "--data_folder", default="../../../../data/modality_matching")
parser.add_argument("--resume", action="store_true")
parser.add_argument("-t", "--subtask", default="openproblems_bmmc_cite_phase2_rna")
parser.add_argument("-device", "--device", default="cuda:1")
parser.add_argument("-cpu", "--cpus", default=1, type=int)
parser.add_argument("-seed", "--rnd_seed", default=rndseed, type=int)
parser.add_argument("--span", default=0.3, type=float)
parser.add_argument("--experiment", type=str, default="test", metavar="E", help="experiment name")
parser.add_argument("--obj", type=str, default="m_elbo_naive_warmup", metavar="O",
                    help="objective to use (default: elbo)")
parser.add_argument(
    "--llik_scaling", type=float, default=1., help="likelihood scaling for cub images/svhn modality when running in"
    "multimodal setting, set as 0 to use default value")
parser.add_argument("--batch_size", type=int, default=1000, metavar="N", help="batch size for data (default: 256)")
parser.add_argument("--epochs", type=int, default=100, metavar="E", help="number of epochs to train (default: 100)")
parser.add_argument("--lr", type=float, default=1e-8, metavar="L", help="learning rate (default: 1e-3)")
parser.add_argument("--latent_dim", type=int, default=10, metavar="L", help="latent dimensionality (default: 20)")
parser.add_argument("--num_hidden_layers", type=int, default=2, metavar="H",
                    help="number of hidden layers in enc and dec (default: 2)")
parser.add_argument("--r_hidden_dim", type=int, default=100, help="number of hidden units in enc/dec for gene")
parser.add_argument("--p_hidden_dim", type=int, default=20,
                    help="number of hidden units in enc/dec for protein/peak")
parser.add_argument("--pre_trained", type=str, default="",
                    help="path to pre-trained model (train from scratch if empty)")
parser.add_argument("--learn_prior", type=bool, default=True, help="learn model prior parameters")
parser.add_argument("--print_freq", type=int, default=0, metavar="f",
                    help="frequency with which to print stats (default: 0)")
parser.add_argument("--deterministic_warmup", type=int, default=50, metavar="W", help="deterministic warmup")

for dataset in datasets:
    try:
        
        args = parser.parse_args(['--subtask',dataset,'--device','cuda','--span','1.0'])
        torch.cuda.empty_cache()
        torch.set_num_threads(args.cpus)
        rndseed = args.rnd_seed
        set_seed(rndseed)
        subtask = args.subtask
        device = args.device
        if "gex2atac" in subtask:
            model_class="rna-dna"
        elif "gex2adt" in subtask:
            model_class="rna-protein"
        else:
            raise RuntimeError("model_class error")
        # Preprocess and load data
        dataset = ModalityMatchingDataset(subtask, root=args.data_folder, preprocess="feature_selection",span=args.span)
        data = dataset.load_data()
        # Set data config
        data.set_config(feature_mod=["mod1", "mod2"], label_mod="mod1", feature_channel_type=["layers", "layers"],
                        feature_channel=["counts", "counts"], label_channel="labels")

        (x_train, y_train), _ = data.get_train_data(return_type="torch")
        (x_test, y_test), labels = data.get_test_data(return_type="torch")

        args.r_dim = x_train.shape[1]
        args.p_dim = y_train.shape[1]

        # model_class = "rna-protein" if subtask == "openproblems_bmmc_cite_phase2_rna" else "rna-dna"
        model = MMVAE(model_class, args).to(device)

        model.fit(x_train, y_train)
        print(model.predict(x_test, y_test))
        score=model.score(x_test, y_test, labels)
    except Exception as e:
        score=e
    finally:
        print(score)
        scMM_scores.append(score)
"""To reproduce scMM on other samples, please refer to command lines belows:

GEX-ADT (subset):
python scmm.py --subtask openproblems_bmmc_cite_phase2_rna_subset --device cuda

GEX-ADT:
python scmm.py --subtask openproblems_bmmc_cite_phase2_rna --device cuda

GEX-ATAC:
python scmm.py --subtask openproblems_bmmc_multiome_phase2_rna --device cuda

"""


[INFO][2023-10-07 14:42:23,002][dance][set_seed] Setting global random seed to 205572460
[INFO][2023-10-07 14:42:23,004][dance][set_seed] Setting global random seed to 205572460
[INFO][2023-10-07 14:42:23,006][dance][set_seed] Setting global random seed to 205572460
[INFO][2023-10-07 14:42:23,008][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/openproblems_2022_cite_gex2adt_subset/openproblems_2022_cite_gex2adt_subset.open_dataset.output_train_mod1.h5ad


model_class error
model_class error


[INFO][2023-10-07 14:42:23,328][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/openproblems_2022_cite_gex2adt_subset/openproblems_2022_cite_gex2adt_subset.open_dataset.output_train_mod2.h5ad
[INFO][2023-10-07 14:42:23,353][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/openproblems_2022_cite_gex2adt_subset/openproblems_2022_cite_gex2adt_subset.open_dataset.output_train_sol.h5ad
[INFO][2023-10-07 14:42:23,407][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/openproblems_2022_cite_gex2adt_subset/openproblems_2022_cite_gex2adt_subset.open_dataset.output_test_mod1.h5ad
[INFO][2023-10-07 14:42:23,689][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/openproblems_2022_cite_gex2adt_subset/openproblems_2022_cite_gex2adt_subset.open_dataset.output_test_mod2.h5ad
[INFO][2023-10-07 14:42:23,716][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/openproblems_2022_cite_gex2adt_subset/openproblems_2022_cite

====> Epoch: 001 Train loss: 0.0000
====>             Valid loss: 154522.0471
====> Epoch: 002 Train loss: 0.0000
====>             Valid loss: 153270.2748
====> Epoch: 003 Train loss: 0.0000
====>             Valid loss: 149404.7073
====> Epoch: 004 Train loss: 0.0000
====>             Valid loss: 148863.2339
====> Epoch: 005 Train loss: 0.0000
====>             Valid loss: 150495.8899
====> Epoch: 006 Train loss: 0.0000
====>             Valid loss: 155663.4767
====> Epoch: 007 Train loss: 0.0000
====>             Valid loss: 154033.2875
====> Epoch: 008 Train loss: 0.0000
====>             Valid loss: 153022.2136
====> Epoch: 009 Train loss: 0.0000
====>             Valid loss: 147634.7784
====> Epoch: 010 Train loss: 0.0000
====>             Valid loss: 147072.8866
Valid Matching score: 0.0
====> Epoch: 011 Train loss: 0.0000
====>             Valid loss: 141123.9191
====> Epoch: 012 Train loss: 0.0000
====>             Valid loss: 153843.3402
====> Epoch: 013 Train loss: 0.0000
==

[INFO][2023-10-07 14:42:36,069][dance][set_seed] Setting global random seed to 205572460
[INFO][2023-10-07 14:42:36,070][dance][set_seed] Setting global random seed to 205572460
[INFO][2023-10-07 14:42:36,071][dance][set_seed] Setting global random seed to 205572460
[INFO][2023-10-07 14:42:36,072][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE117089_mouse_gex2atac/GSE117089_mouse_gex2atac.GSE117089_dataset.output_train_mod1.h5ad
[INFO][2023-10-07 14:42:36,202][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE117089_mouse_gex2atac/GSE117089_mouse_gex2atac.GSE117089_dataset.output_train_mod2.h5ad


...

[INFO][2023-10-07 14:43:29,188][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE127064_p0Brain_gex2atac/GSE127064_p0Brain_gex2atac.GSE126074_dataset.output_train_sol.h5ad
[INFO][2023-10-07 14:43:29,202][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE127064_p0Brain_gex2atac/GSE127064_p0Brain_gex2atac.GSE126074_dataset.output_test_mod1.h5ad
[INFO][2023-10-07 14:43:29,241][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE127064_p0Brain_gex2atac/GSE127064_p0Brain_gex2atac.GSE126074_dataset.output_test_mod2.h5ad
[INFO][2023-10-07 14:43:29,577][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE127064_p0Brain_gex2atac/GSE127064_p0Brain_gex2atac.GSE126074_dataset.output_test_sol.h5ad
[INFO][2023-10-07 14:43:32,493][dance][_maybe_preprocess] Preprocessing done.
[INFO][2023-10-07 14:43:32,654][dance][load_data] Raw data loaded:
Data object that wraps (.data):
MuData object with n_obs × n_vars = 5081 × 20000
  u

====> Epoch: 001 Train loss: 0.0000
====>             Valid loss: 1098.3858
====> Epoch: 002 Train loss: 0.0000
====>             Valid loss: 1099.5620
====> Epoch: 003 Train loss: 0.0000
====>             Valid loss: 1099.6898
====> Epoch: 004 Train loss: 0.0000
====>             Valid loss: 1098.9435
====> Epoch: 005 Train loss: 0.0000
====>             Valid loss: 1098.9509
====> Epoch: 006 Train loss: 0.0000
====>             Valid loss: 1098.9066
====> Epoch: 007 Train loss: 0.0000
====>             Valid loss: 1099.4226
====> Epoch: 008 Train loss: 0.0000
====>             Valid loss: 1098.9782
====> Epoch: 009 Train loss: 0.0000
====>             Valid loss: 1098.3701
====> Epoch: 010 Train loss: 0.0000
====>             Valid loss: 1107.7303
Valid Matching score: 0.0
====> Epoch: 011 Train loss: 0.0000
====>             Valid loss: 1098.5233
====> Epoch: 012 Train loss: 0.0000
====>             Valid loss: 1098.8537
====> Epoch: 013 Train loss: 0.0000
====>             Valid lo

[INFO][2023-10-07 14:43:39,236][dance][set_seed] Setting global random seed to 205572460
[INFO][2023-10-07 14:43:39,237][dance][set_seed] Setting global random seed to 205572460
[INFO][2023-10-07 14:43:39,238][dance][set_seed] Setting global random seed to 205572460
[INFO][2023-10-07 14:43:39,239][dance][set_seed] Setting global random seed to 205572460
[INFO][2023-10-07 14:43:39,240][dance][set_seed] Setting global random seed to 205572460
[INFO][2023-10-07 14:43:39,241][dance][set_seed] Setting global random seed to 205572460
[INFO][2023-10-07 14:43:39,242][dance][set_seed] Setting global random seed to 205572460


model_class error
model_class error
model_class error
model_class error
model_class error
model_class error
model_class error


'To reproduce scMM on other samples, please refer to command lines belows:\n\nGEX-ADT (subset):\npython scmm.py --subtask openproblems_bmmc_cite_phase2_rna_subset --device cuda\n\nGEX-ADT:\npython scmm.py --subtask openproblems_bmmc_cite_phase2_rna --device cuda\n\nGEX-ATAC:\npython scmm.py --subtask openproblems_bmmc_multiome_phase2_rna --device cuda\n\n'

In [6]:
scMM_scores

[RuntimeError('model_class error'),
 RuntimeError('model_class error'),
 0.0,
 RuntimeError('model_class error'),
 RuntimeError('model_class error'),
 0.0002950722991954535,
 0.0013812155229970813,
 0.0,
 0.0006557377055287361,
 RuntimeError('model_class error'),
 RuntimeError('model_class error'),
 RuntimeError('model_class error'),
 RuntimeError('model_class error'),
 RuntimeError('model_class error'),
 RuntimeError('model_class error'),
 RuntimeError('model_class error')]

In [8]:
ScMoGNN_scores=[]
import argparse
import random

import numpy as np
import torch
import torch.nn.functional as F

from dance.datasets.multimodality import ModalityMatchingDataset
from dance.modules.multi_modality.match_modality.scmogcn import ScMoGCNWrapper
from dance.transforms.graph.cell_feature_graph import CellFeatureBipartiteGraph
from dance.utils import set_seed
import scanpy as sc

rndseed = random.randint(0, 2147483647)
parser = argparse.ArgumentParser()
parser.add_argument("-t", "--subtask", default="openproblems_bmmc_cite_phase2_rna")
parser.add_argument("-d", "--data_folder", default="../../../../data/modality_matching")
parser.add_argument("-csv", "--csv_path", default="decoupled_lsi.csv")
parser.add_argument("-l", "--layers", default=4, type=int, choices=[3, 4, 5, 6, 7])
parser.add_argument("-lr", "--learning_rate", default=6e-4, type=float)
parser.add_argument("-dis", "--disable_propagation", default=0, type=int, choices=[0, 1, 2])
parser.add_argument("-aux", "--auxiliary_loss", default=True, type=bool)
parser.add_argument("-pk", "--pickle_suffix", default="_lsi_input_pca_count.pkl")
parser.add_argument("-seed", "--rnd_seed", default=rndseed, type=int)
parser.add_argument("-cpu", "--cpus", default=1, type=int)
parser.add_argument("-device", "--device", default="cuda:2")
parser.add_argument("-e", "--epochs", default=200, type=int)
parser.add_argument("-tq", "--threshold_quantile", default=0.95, type=float)
parser.add_argument("--span", default=0.3, type=float)

for dataset in datasets:
    torch.cuda.empty_cache()
    args = parser.parse_args(['--subtask',dataset,'--device','cuda'])

    torch.set_num_threads(args.cpus)
    rndseed = args.rnd_seed
    set_seed(rndseed)

    subtask = args.subtask
    data_folder = args.data_folder
    device = args.device
    layers = args.layers
    pkl_path = subtask + args.pickle_suffix

    # Preprocess and load data
    dataset = ModalityMatchingDataset(subtask, root=data_folder, preprocess="pca", pkl_path=pkl_path,span=args.span)


    
    data = dataset.load_data()
    sc.pp.log1p(data.mod["mod1"])
    sc.pp.log1p(data.mod["mod2"])
    # ScMoGNN graph construction
    data = CellFeatureBipartiteGraph(cell_feature_channel="X_pca", mod="mod1")(data)
    data = CellFeatureBipartiteGraph(cell_feature_channel="X_pca", mod="mod2")(data)

    data.set_config(feature_mod=["mod1", "mod2", "mod1", "mod2"], feature_channel_type=["uns", "uns", "obs", "obs"],
                    feature_channel=["g", "g", "batch", "batch"], label_mod="mod1", label_channel="labels")
    (g_mod1, g_mod2, batch_mod1, batch_mod2), z = data.get_data(return_type="default")

    
    if subtask == "openproblems_bmmc_cite_phase2_rna":
        HIDDEN_SIZE = 64
        TEMPERATURE = 2.739896
        model = ScMoGCNWrapper(
            args,
            [
                [(g_mod1.num_nodes("feature"), 512, 0.25), (512, 512, 0.25), (512, HIDDEN_SIZE)],
                [(g_mod2.num_nodes("feature"), 512, 0.2), (512, 512, 0.2), (512, HIDDEN_SIZE)],
                [(HIDDEN_SIZE, 512, 0.2), (512, g_mod1.num_nodes("feature"))],
                [(HIDDEN_SIZE, 512, 0.2), (512, g_mod2.num_nodes("feature"))],
            ],
            TEMPERATURE,
        )  # yapf: disable
    else:
        HIDDEN_SIZE = 256
        TEMPERATURE = 3.065016
        model = ScMoGCNWrapper(
            args,
            [
                [(g_mod1.num_nodes("feature"), 1024, 0.5), (1024, 1024, 0.5), (1024, HIDDEN_SIZE)],
                [(g_mod2.num_nodes("feature"), 2048, 0.5), (2048, HIDDEN_SIZE)],
                [(HIDDEN_SIZE, 512, 0.2), (512, g_mod1.num_nodes("feature"))],
                [(HIDDEN_SIZE, 512, 0.2), (512, g_mod2.num_nodes("feature"))],
            ],
            TEMPERATURE,
        )  # yapf: disable

    train_size = len(data.get_split_idx("train"))
    z_test = F.one_hot(torch.from_numpy(z[train_size:]).long())
    labels1 = torch.argmax(z_test, dim=0).to(device)
    labels2 = torch.argmax(z_test, dim=1).to(device)
    g_mod1 = g_mod1.to(device)
    g_mod2 = g_mod2.to(device)

    model.fit(g_mod1, g_mod2, labels1, labels2, train_size=train_size)
    model.load(f"models/model_{rndseed}.pth")

    test_idx = np.arange(train_size, g_mod1.num_nodes("cell"))
    pred = model.predict(test_idx, enhance=False, batch1=batch_mod1, batch2=batch_mod2,
                         threshold_quantile=args.threshold_quantile)
    score = model.score(test_idx, labels1=labels1,labels2=labels2,enhance=False, batch1=batch_mod1, batch2=batch_mod2,
                        threshold_quantile=args.threshold_quantile)

    print(pred)
    print(score)
    ScMoGNN_scores.append(score)
"""To reproduce scMoGCN on other samples, please refer to command lines belows:

GEX-ADT (subset):
python scmogcn.py --subtask openproblems_bmmc_cite_phase2_rna_subset --threshold_quantile 0.85 --device cuda

GEX-ADT:
python scmogcn.py --subtask openproblems_bmmc_cite_phase2_rna --device cuda

GEX-ATAC:
python scmogcn.py --subtask openproblems_bmmc_multiome_phase2_rna --device cuda

"""


[INFO][2023-10-07 20:09:46,472][dance][set_seed] Setting global random seed to 1114053323
[INFO][2023-10-07 20:09:46,476][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/pbmc_cite/pbmc_cite.citeanti_dataset.output_train_mod1.h5ad
[INFO][2023-10-07 20:09:47,420][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/pbmc_cite/pbmc_cite.citeanti_dataset.output_train_mod2.h5ad
[INFO][2023-10-07 20:09:47,480][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/pbmc_cite/pbmc_cite.citeanti_dataset.output_train_sol.h5ad
[INFO][2023-10-07 20:09:47,547][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/pbmc_cite/pbmc_cite.citeanti_dataset.output_test_mod1.h5ad
[INFO][2023-10-07 20:09:47,916][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/pbmc_cite/pbmc_cite.citeanti_dataset.output_test_mod2.h5ad
[INFO][2023-10-07 20:09:47,942][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/pbmc_cite/pbmc_cite

<class 'scipy.sparse._coo.coo_matrix'>
<class 'scipy.sparse._coo.coo_matrix'>


[INFO][2023-10-07 20:11:16,106][dance][_maybe_preprocess] Preprocessing done.


<class 'scipy.sparse._coo.coo_matrix'>
<class 'scipy.sparse._coo.coo_matrix'>


/home/zyxing/anaconda3/envs/dance/lib/python3.8/site-packages/mudata/_core/mudata.py:491: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(
/home/zyxing/anaconda3/envs/dance/lib/python3.8/site-packages/mudata/_core/mudata.py:1059: UserWarning: Modality names will be prepended to var_names since there are identical var_names in different modalities.
  warnings.warn(
[INFO][2023-10-07 20:11:28,532][dance][load_data] Raw data loaded:
Data object that wraps (.data):
MuData object with n_obs × n_vars = 33454 × 17034
  uns:	'dance_config'
  2 modalities
    mod1:	33454 x 17009
      obs:	'batch'
      obsm:	'X_pca', 'labels'
      layers:	'counts'
    mod2:	33454 x 25
      obs:	'batch'
      obsm:	'X_pca'
      layers:	'counts'
[INFO][2023-10-07 20:11:28,536][dance][wrapped_func] Took 0:01:42.061835 to load and process data.
[INFO][2023-10-07 20:11:29,303][dance][set_config_from_dict] Setting config 'feature_mod' to ['mod1', 'mod2', 'mo

...

[INFO][2023-10-07 20:40:49,180][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_train_mod2.h5ad
[INFO][2023-10-07 20:40:49,224][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_train_sol.h5ad
[INFO][2023-10-07 20:40:49,237][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_test_mod1.h5ad
[INFO][2023-10-07 20:40:49,509][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_test_mod2.h5ad
[INFO][2023-10-07 20:40:49,551][dance][_load_raw_data] Loading /home/zyxing/data/modality_matching/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_test_sol.h5ad
/home/zyxing/dance/dance/transforms/preprocess.py:154: RuntimeWar

<class 'scipy.sparse._coo.coo_matrix'>
<class 'scipy.sparse._coo.coo_matrix'>
<class 'scipy.sparse._coo.coo_matrix'>
<class 'scipy.sparse._coo.coo_matrix'>


[INFO][2023-10-07 20:41:24,655][dance][_maybe_preprocess] Preprocessing done.
[INFO][2023-10-07 20:41:25,552][dance][load_data] Raw data loaded:
Data object that wraps (.data):
MuData object with n_obs × n_vars = 1365 × 297552
  uns:	'dance_config'
  2 modalities
    mod1:	1365 x 286071
      obs:	'batch'
      obsm:	'X_pca', 'labels'
      layers:	'counts'
    mod2:	1365 x 11481
      obs:	'batch'
      obsm:	'X_pca'
      layers:	'counts'
[INFO][2023-10-07 20:41:25,553][dance][wrapped_func] Took 0:00:36.703964 to load and process data.
[INFO][2023-10-07 20:41:25,585][dance][set_config_from_dict] Setting config 'feature_mod' to ['mod1', 'mod2', 'mod1', 'mod2']
[INFO][2023-10-07 20:41:25,585][dance][set_config_from_dict] Setting config 'feature_channel_type' to ['uns', 'uns', 'obs', 'obs']
[INFO][2023-10-07 20:41:25,586][dance][set_config_from_dict] Setting config 'feature_channel' to ['g', 'g', 'batch', 'batch']
[INFO][2023-10-07 20:41:25,587][dance][set_config_from_dict] Setting conf

tensor([[3.9332, 2.2481, 2.2481,  ..., 2.2481, 2.2481, 2.2079],
        [3.6482, 2.2415, 2.2415,  ..., 2.2415, 2.2415, 2.2580],
        [3.6482, 2.2415, 2.2415,  ..., 2.2415, 2.2415, 2.2580],
        ...,
        [3.6482, 2.2415, 2.2415,  ..., 2.2415, 2.2415, 2.2580],
        [3.6482, 2.2415, 2.2415,  ..., 2.2415, 2.2415, 2.2580],
        [3.6482, 2.2415, 2.2415,  ..., 2.2415, 2.2415, 2.2580]],
       device='cuda:0')
0.0012195121962577105


'To reproduce scMoGCN on other samples, please refer to command lines belows:\n\nGEX-ADT (subset):\npython scmogcn.py --subtask openproblems_bmmc_cite_phase2_rna_subset --threshold_quantile 0.85 --device cuda\n\nGEX-ADT:\npython scmogcn.py --subtask openproblems_bmmc_cite_phase2_rna --device cuda\n\nGEX-ATAC:\npython scmogcn.py --subtask openproblems_bmmc_multiome_phase2_rna --device cuda\n\n'

In [9]:
ScMoGNN_scores

[0.007970509119331837,
 0.0029999999096617103,
 0.00029999999242136255,
 0.00033333332976326346,
 0.0004999999946448952,
 0.0013278253463795409,
 0.0017265193746425211,
 0.00048496603267267346,
 0.0009836065582931042,
 0.00159744406118989,
 0.0012562813935801387,
 0.001775147917214781,
 0.0024558608420193195,
 0.0006922011816641316,
 0.0002691065601538867,
 0.0012195121962577105]